This notebook reads in the conbined SNe catalog file ('1.FullCatalog.csv') and searches through PHANGS CO map images, looking for OSC objects that have gone off since a chosen year. 


In [1]:
#import packages and formatting statements

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import ascii

plt.rc('text',usetex=False)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

In [2]:
# assign previous and future year boundaries for survey, list actual cutoff year for filename

PREVIOUS_YEAR = "2018"
FUTURE_YEAR = "2023"
CUTOFF_YEAR = "2019"

In [4]:
FullCat = '../Data/1.FullCatalog.csv'
SNTable = Table.read(FullCat, format='csv')
SNTable

Name,Type,Host,RA hms,Dec dms,RA dds,Dec dds,DiscDate
str36,str19,str44,str11,str12,float64,float64,str23
2018inf,Ia,MCG +07-28-26,13:31:56.1,+44:08:21.69,202.98375000000001,44.139358333333334,2018/11/15
A2021ymb,Ia,--,0:20:51.16,-32:51:24.88,5.21315,-32.85691111111111,2021/09/08
ASASSN-13av,Ia,NGC 7068,21:26:31.92,+12:10:48.72,321.633,12.1802,2013/06/26
ASASSN-13ch,Ia,CGCG 023-030,16:16:34.07,-0:35:27.36,244.14195833333332,-0.5909322222222222,2013/08/28
ASASSN-13cj,Ia,CGCG 051-075,16:17:11.03,+4:33:14.69,244.29593749999998,4.554081666666667,2013/08/27
ASASSN-13co,II,CGCG 402-014,21:40:38.63,+6:30:36.71,325.1609444444445,6.510197222222222,2013/08/29
...,...,...,...,...,...,...,...
iPTF16fnl,Unclassified,--,0:29:57.1,+32:53:38.4,7.487916666666667,32.894,2016
iPTF16fsc,Ia,--,2:24:28.34,+32:51:30.69,36.11808333333333,32.858525,2016/09/04


In [5]:
# shorten DiscDate column to only have year

DiscYear = [c[0:4] for c in SNTable["DiscDate"]]
SNTable.add_column(DiscYear, name="DiscYear")


In [6]:
# remove rows of SNTable that have year < year chosen

mask = (SNTable["DiscYear"] > PREVIOUS_YEAR) & (SNTable["DiscYear"] < FUTURE_YEAR) 
SNRecent = SNTable[mask] 

SNRecent


Name,Type,Host,RA hms,Dec dms,RA dds,Dec dds,DiscDate,DiscYear
str36,str19,str44,str11,str12,float64,float64,str23,str4
A2021ymb,Ia,--,0:20:51.16,-32:51:24.88,5.21315,-32.85691111111111,2021/09/08,2021
ASASSN-19abg,Unclassified,IC 4890,19:45:32.16,-56:33:11.52,296.384,-56.5532,2019/11/15,2019
ASASSN-19kb,Ia,LEDA 986847,12:45:51.07,-9:45:21.15,191.4627847222222,-9.755873888888887,2019/04/13,2019
ASASSN-19kz,II,NGC 2207,6:16:22.25,-21:21:35.47,94.09271666666666,-21.359853703703703,2019/04/27,2019
ASASSN-19lp,II,2MASX J22522926-1105320,22:52:29.3,-11:05:25.64,343.12210208333335,-11.090454629629631,2019/05/02,2019
ASASSN-19nb,Ia,GALEXASC J231739.47-591551.0,23:17:38.76,-59:15:42.39,349.4114861111111,-59.261775925925924,2019/05/16,2019
...,...,...,...,...,...,...,...,...
SNhunt431,Unclassified,MCG +01-30-19,11:54:54.29,+2:58:37.74,178.7262,2.97715,2020/04/23,2020
SNhunt432,Unclassified,UGC 8588,13:35:41.57,+45:55:31.8,203.9232,45.9255,2020/05/05,2020


In [7]:
fileName = "../Data/2.RecentSNTable" + CUTOFF_YEAR + ".csv"
SNRecent.write(fileName, overwrite = True)
